## INF1608 - Análise Numérica - 2017.1
## Departamento de Informática - PUC-Rio 
## Prof. Hélio Lopes - lopes@inf.puc-rio.br
## http://www.inf.puc-rio.br/~lopes


## Lista 5

1) Implemente o método de Reflexão de Householder para a obtenção da decomposição QR de uma matriz quadrada nxn.
       
2) Implemente o método de Rotação de Givens para a obtenção da decomposição QR de uma matriz quadrada nxn.
     
3) Teste muito bem todos esses algoritmos!

In [13]:
#1)
import numpy as np

# Transforma um array em uma matrix coluna
def columnVector(v):
    return np.matrix(v).transpose()

# Monta a matriz Qk a partir de Qk' e o tamanho desejado
def mountQk(Ql, n):
    Q = np.zeros(shape=(n,n))
    k = n - len(Ql)
    for i in range(n):
        for j in range(n):
            if (i < k or j < k):
                if (i == j):
                    Q[i][j] = 1
                else:
                    Q[i][j] = 0
            else:
                Q[i][j] = Ql[i-k][j-k]
    return Q

def householderDecomp(A):
    m = len(A)
    n = len(A[0])
    e1 = np.zeros(m)
    e1[0] = 1.
    Q = np.identity(m)
    R = A.copy()
    k = 0
    Al = A.copy() # Al significa A'
    QlSize = m # valor do lado da matriz quadrada Q'
    while(k < m - 1 and k < n): # k até min(m-1, n)
        x = Al[:,0]
        alpha = np.linalg.norm(x)
        u = x - alpha * e1
        v = u/np.linalg.norm(u)
        v = columnVector(v) # correção para cálculos
        Ql = np.identity(QlSize) - 2 * v.dot(v.transpose()) # Ql significa Q'
        Ql = np.array(Ql) # correção para cálculos
        
        # Atualizar Q e R através de Qk (R = Qj*..*Q1*Q0*A; Q = Q0t*Q1t*..*Qjt)
        Qk = mountQk(Ql, m)
        Q = Q.dot(Qk.transpose())
        R = Qk.dot(R)
        
        # Preparação da próxima iteração:
        k += 1
        
        # Calcular nova A':
        QlSize = m - k
        Al = Ql.dot(Al)
        Al = Al[1:QlSize+1,1:QlSize+1]
        
        # Atualizar e1 para a dimensão correta:
        e1 = np.zeros(QlSize)
        e1[0] = 1.
    return Q, R

A = np.array([[12,-51,4],[6,167,-68],[-4,24,-41]])
print("A:\n" + str(A))
Q, R = householderDecomp(A)
print ("Q:\n" + str(Q) + "\nR:\n" + str(R) + "\nQR:\n" + str(Q.dot(R)))

A:
[[ 12 -51   4]
 [  6 167 -68]
 [ -4  24 -41]]
Q:
[[ 0.85714286 -0.39428571  0.33142857]
 [ 0.42857143  0.90285714 -0.03428571]
 [-0.28571429  0.17142857  0.94285714]]
R:
[[  1.40000000e+01   2.10000000e+01  -1.40000000e+01]
 [  5.50670620e-16   1.75000000e+02  -7.00000000e+01]
 [ -3.01980663e-16  -9.23705556e-16  -3.50000000e+01]]
QR:
[[  12.  -51.    4.]
 [   6.  167.  -68.]
 [  -4.   24.  -41.]]


In [14]:
#2)
import numpy as np

# Recebe um vetor (x, y) e retorna seu ângulo anti-horário com o sentido positivo do eixo x
def getAngle2D(x, y):
    norm = np.sqrt(x*x + y*y)
    xn = x/norm
    ang = np.arccos(xn)
    if (y >= 0):
        return ang
    else:
        return 2 * np.pi - ang

# Monta a matriz Qk "n x n" a partir do ângulo de rotação desejado e a posição i, j de A que se deseja zerar
def mountQk(n, theta, i, j):
    Qk = np.zeros(shape=(n,n))
    for p in range(n):
        if (p != i and p != j):
            Qk[p][p] = 1
    Qk[j][j] = np.cos(theta)
    Qk[j][i] = -np.sin(theta)
    Qk[i][j] = np.sin(theta)
    Qk[i][i] = np.cos(theta)
    return Qk

def givensDecomp(A):
    m = len(A)
    n = len(A[0])
    Q = np.identity(m)
    R = A.copy()
    j = 0
    while(j < m - 1 and j < n): # j até min(m-1, n)
        for i in range(j + 1, m):
            theta = 2 * np.pi - getAngle2D(R[j][j], R[i][j]) # Angulo de rotação desejado para zerar Qk*..*Q1*Q0*A[i][j]
            Qk = mountQk(m, theta, i, j)
            
            # Atualizar Q e R através de Qk (R = Qj*..*Q1*Q0*A; Q = Q0t*Q1t*..*Qjt)
            Q = Q.dot(Qk.transpose())
            R = Qk.dot(R)
        # Preparar próxima iteração
        j += 1
            
    return Q, R

A = np.array([[12,-51,4],[6,167,-68],[-4,24,-41]])
print("A:\n" + str(A))
Q, R = givensDecomp(A)
print ("Q:\n" + str(Q) + "\nR:\n" + str(R) + "\nQR:\n" + str(Q.dot(R)))

A:
[[ 12 -51   4]
 [  6 167 -68]
 [ -4  24 -41]]
Q:
[[ 0.85714286 -0.39428571  0.33142857]
 [ 0.42857143  0.90285714 -0.03428571]
 [-0.28571429  0.17142857  0.94285714]]
R:
[[  1.40000000e+01   2.10000000e+01  -1.40000000e+01]
 [ -2.62155607e-15   1.75000000e+02  -7.00000000e+01]
 [  5.44171486e-15   6.40255177e-15  -3.50000000e+01]]
QR:
[[  12.  -51.    4.]
 [   6.  167.  -68.]
 [  -4.   24.  -41.]]


In [ ]:
#3)
Testes executados nas qeustões específicas, resultado comparado com o exemplo apresentado pelo wikipedia no link
https://en.wikipedia.org/wiki/QR_decomposition
Os testes tiveram sucesso no resultado obtido, tal como esperado.